##

## DACON 운동 동작 분류 AI 경진대회
- 3축 가속도계(accelerometer)와 3축 자이로스코프(gyroscope)를 활용해 측정된 센서 데이터에 머신러닝 알고리즘을 적용해 운동 동작 인식 알고리즘 개발
- 대회 [참조](https://dacon.io/competitions/official/235689/overview/) 

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# 데이터가 있는 폴더
data_dir = 'D://DACON/11_exercise/data'


# 데이터 불러오기
train = pd.read_csv('{data_dir}/train_features.csv'.format(data_dir=data_dir))
train_labels = pd.read_csv('{data_dir}/train_labels.csv'.format(data_dir=data_dir))
test = pd.read_csv('{data_dir}/test_features.csv'.format(data_dir=data_dir))
submission = pd.read_csv('{data_dir}/sample_submission.csv'.format(data_dir=data_dir))


In [3]:
print('train 데이터셋의 크기: {0}'.format(train.shape))
print('train labels의 크기: {0}'.format(train_labels.shape))
print('test 데이터셋의 크기: {0}'.format(test.shape))
print('test estimation의 크기: {0}'.format(submission.shape))

train 데이터셋의 크기: (1875000, 8)
train labels의 크기: (3125, 3)
test 데이터셋의 크기: (469200, 8)
test estimation의 크기: (782, 62)


- train에 187.5만개의 레코드가 존재하나, labels은 3125개 밖에 존재하지 않음
train에 187.5만개의 레코드가 존재하나, labels은 3125개 밖에 존재하지 않음
- 확인이 필요

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875000 entries, 0 to 1874999
Data columns (total 8 columns):
 #   Column  Dtype  
---  ------  -----  
 0   id      int64  
 1   time    int64  
 2   acc_x   float64
 3   acc_y   float64
 4   acc_z   float64
 5   gy_x    float64
 6   gy_y    float64
 7   gy_z    float64
dtypes: float64(6), int64(2)
memory usage: 114.4 MB


In [5]:
train_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3125 entries, 0 to 3124
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3125 non-null   int64 
 1   label       3125 non-null   int64 
 2   label_desc  3125 non-null   object
dtypes: int64(2), object(1)
memory usage: 73.4+ KB


In [6]:
train[:5]

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234


In [7]:
train_labels[:5]

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise


- train 데이터셋의 id가 여러개 중복되는 것을 확인할 수 있음

In [8]:
train.groupby('id')['time'].count()

id
0       600
1       600
2       600
3       600
4       600
       ... 
3120    600
3121    600
3122    600
3123    600
3124    600
Name: time, Length: 3125, dtype: int64

In [9]:
train.groupby('id').count()[train['id'].value_counts() != 600]

,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
id,,,,,,,


- 총 id별로 600개씩, 3125개의 데이터를 가지고 있는 것을 확인할 수 있음

In [10]:
# train labels 운동 갯수
print('운동 동작 갯수: {0}'.format(len(train_labels['label'].unique())))
unique_labels = np.sort(train_labels['label'].unique())

운동 동작 갯수: 61


### 데이터셋 설명
- train: 600*3125개의 3축 가속도계(accelerometer)와 3축 자이로스코프(gyroscope)를 활용해 측정된 센서 데이터
- train_labels: 3125개의 id별 동작과 동작 label(61개 동작)
- test: 600*782개의 가속도계, 자이로스코프 센서 데이터
- submission: 782개의 동작 예측 결과 (782*62)

In [11]:
train[:10]

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
5,0,5,1.289304,-0.214665,-0.249887,10.065865,-64.250636,-29.286448
6,0,6,1.281405,-0.265540,-0.257836,3.916651,-68.661900,-27.142303
7,0,7,1.242273,-0.292931,-0.269638,5.153811,-73.190911,-29.084766
8,0,8,1.198871,-0.273369,-0.292713,8.494859,-74.849794,-31.135732
9,0,9,1.113677,-0.253454,-0.230331,22.405416,-85.755582,-37.407136


In [12]:
train_merged = pd.merge(train,train_labels, on='id', how='outer')
train_merged

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,label,label_desc
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,37,Shoulder Press (dumbbell)
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,37,Shoulder Press (dumbbell)
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,37,Shoulder Press (dumbbell)
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,37,Shoulder Press (dumbbell)
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,37,Shoulder Press (dumbbell)
...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,2,Bicep Curl
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,2,Bicep Curl
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,2,Bicep Curl
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,2,Bicep Curl


In [13]:
train_merged

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,label,label_desc
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,37,Shoulder Press (dumbbell)
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,37,Shoulder Press (dumbbell)
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,37,Shoulder Press (dumbbell)
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,37,Shoulder Press (dumbbell)
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,37,Shoulder Press (dumbbell)
...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,2,Bicep Curl
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,2,Bicep Curl
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,2,Bicep Curl
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,2,Bicep Curl


In [14]:
# 각각의 label이 골고루 분포 되어있는지 확인
dict_lab_dist = {label : train_labels[train_labels.label == label].shape[0] for label in unique_labels}
dict_lab_dist

{0: 12,
 1: 21,
 2: 20,
 3: 23,
 4: 35,
 5: 25,
 6: 24,
 7: 26,
 8: 97,
 9: 37,
 10: 20,
 11: 23,
 12: 12,
 13: 12,
 14: 25,
 15: 25,
 16: 22,
 17: 27,
 18: 47,
 19: 20,
 20: 26,
 21: 27,
 22: 19,
 23: 20,
 24: 35,
 25: 24,
 26: 1518,
 27: 34,
 28: 55,
 29: 20,
 30: 35,
 31: 20,
 32: 18,
 33: 20,
 34: 22,
 35: 30,
 36: 28,
 37: 35,
 38: 20,
 39: 20,
 40: 34,
 41: 20,
 42: 20,
 43: 35,
 44: 21,
 45: 22,
 46: 20,
 47: 26,
 48: 25,
 49: 30,
 50: 37,
 51: 24,
 52: 12,
 53: 13,
 54: 23,
 55: 37,
 56: 36,
 57: 20,
 58: 20,
 59: 23,
 60: 48}

- 26번 운동에 대한 데이터에 지나치게 편중되어 있음
26번 운동에 대한 데이터에 지나치게 편중되어 있음
- 26번 운동이 무엇인지 확인

In [15]:
train_labels[train_labels['label'] == 26]

,id,label,label_desc
1,1,26,Non-Exercise
3,3,26,Non-Exercise
4,4,26,Non-Exercise
5,5,26,Non-Exercise
6,6,26,Non-Exercise
...,...,...,...
3114,3114,26,Non-Exercise
3116,3116,26,Non-Exercise
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise


In [16]:
# 전체 데이터 non-exercise의 비율
ratio_non_workout = dict_lab_dist[26]/sum(list(dict_lab_dist.values())) * 100
print('전체 데이터 중 non-exercise의 비율: {0:.4f}'.format(ratio_non_workout))

전체 데이터 중 non-exercise의 비율: 48.5760


In [17]:
# 61개의 labels 있으니, 다른 label의 비율은 1% 되지 않을 정도로 매우 작을 것으로 예상됨
# 다른 데이터의 비율을 찾아봄
for num in list(dict_lab_dist.keys()):
    ratio_non_workout = dict_lab_dist[num]/sum(list(dict_lab_dist.values())) * 100
    print('전체 데이터 중 {1}의 비율: {0:.4f}'.format(ratio_non_workout, num))

전체 데이터 중 0의 비율: 0.3840
전체 데이터 중 1의 비율: 0.6720
전체 데이터 중 2의 비율: 0.6400
전체 데이터 중 3의 비율: 0.7360
전체 데이터 중 4의 비율: 1.1200
전체 데이터 중 5의 비율: 0.8000
전체 데이터 중 6의 비율: 0.7680
전체 데이터 중 7의 비율: 0.8320
전체 데이터 중 8의 비율: 3.1040
전체 데이터 중 9의 비율: 1.1840
전체 데이터 중 10의 비율: 0.6400
전체 데이터 중 11의 비율: 0.7360
전체 데이터 중 12의 비율: 0.3840
전체 데이터 중 13의 비율: 0.3840
전체 데이터 중 14의 비율: 0.8000
전체 데이터 중 15의 비율: 0.8000
전체 데이터 중 16의 비율: 0.7040
전체 데이터 중 17의 비율: 0.8640
전체 데이터 중 18의 비율: 1.5040
전체 데이터 중 19의 비율: 0.6400
전체 데이터 중 20의 비율: 0.8320
전체 데이터 중 21의 비율: 0.8640
전체 데이터 중 22의 비율: 0.6080
전체 데이터 중 23의 비율: 0.6400
전체 데이터 중 24의 비율: 1.1200
전체 데이터 중 25의 비율: 0.7680
전체 데이터 중 26의 비율: 48.5760
전체 데이터 중 27의 비율: 1.0880
전체 데이터 중 28의 비율: 1.7600
전체 데이터 중 29의 비율: 0.6400
전체 데이터 중 30의 비율: 1.1200
전체 데이터 중 31의 비율: 0.6400
전체 데이터 중 32의 비율: 0.5760
전체 데이터 중 33의 비율: 0.6400
전체 데이터 중 34의 비율: 0.7040
전체 데이터 중 35의 비율: 0.9600
전체 데이터 중 36의 비율: 0.8960
전체 데이터 중 37의 비율: 1.1200
전체 데이터 중 38의 비율: 0.6400
전체 데이터 중 39의 비율: 0.6400
전체 데이터 중 40의 비율: 1.0880
전체 데이터 중 41의 비율: 0.6400
전

In [18]:
# 평균적으로 다른 label은 몇 개의 데이터가 있는가?
list_avg_labels = list(dict_lab_dist.values())
list_avg_labels.pop(26)
int_lab_mean = np.mean(list_avg_labels)
print('non-exercise를 제외한 labels 평균 rows : %.4f'%int_lab_mean)

non-exercise를 제외한 labels 평균 rows : 26.7833


- 약 50% 정도의 데이터가 non-exercise (다른 데이터 들은 0.3 ~ 2% 분포를 보임)
- 아무 운동도 하지 않는 경우의 데이터가 큰 비중을 차지함(=imbalanced data)
- 정확도(accuracy)는 높지만 재현율(recall rate)가 낮게 예측될 가능성이 높아짐
- sampling 방법을 적용하여 재현율을 높이는 방법을 고려해야함
약 50% 정도의 데이터가 non-exercise (다른 데이터 들은 0.3 ~ 2% 분포를 보임)
- 아무 운동도 하지 않는 경우의 데이터가 큰 비중을 차지함(=imbalanced data)
- 정확도(accuracy)는 높지만 재현율(recall rate)가 낮게 예측될 가능성이 높아짐
- sampling 방법을 적용하여 재현율을 높이는 방법을 고려해야함
- 이 baseline에서는 기본적 undersampling 기법으로 추출 후 학습 진행

### undersampling을 위한 라이브러리 설치
- conda
    * conda install -c conda-forge imbalanced-learn 
- pip
    * pip install -U imbalanced-learn

- 현재는 random으로 뽑아냄

In [19]:
import random 

# label == 26인 id array
list_label26 = train_labels[train_labels.label == 26].id.values.tolist()
# 26개(=다른 label의 데이터수의 평균) 랜덤 추출
list_random_sample_26 = random.sample(list_label26,26)

In [20]:
train_not_26 = train_merged[train_merged.label != 26]

In [21]:
train_26 = train_merged[train_merged.id.isin(list_random_sample_26)]

In [22]:
train_26.shape

(15600, 10)

In [23]:
train_not_26.shape

(964200, 10)

In [24]:
X_train = pd.concat([train_not_26,train_26])

In [25]:
X_train.shape

(979800, 10)

In [26]:
# 학습을 위한 라이브러리 불러오기
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

X = tf.reshape(np.array(X_train.iloc[:,2:-2]), [-1,600,6])
X.shape

Using TensorFlow backend.


TensorShape([1633, 600, 6])

In [27]:
# 학습 가능하도록 id,label unique하게 뽑음
y = X_train[['id','label']].iloc[::600,:]
y = tf.keras.utils.to_categorical(y['label']) 
y.shape

(1633, 61)

### 모델 학습 및 예측

In [28]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(600,6)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(X,y, epochs=30, steps_per_epoch=128)

ValueError: If steps_per_epoch is set, the `batch_size` must be None.

In [ ]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 6])
test_X.shape

In [ ]:
prediction=model.predict(test_X)

In [ ]:
prediction.shape

In [ ]:
submission

In [ ]:
submission.iloc[:,1:]=prediction

In [ ]:
submission

In [ ]:
submission.to_csv('baseline_submission.csv', index=False)